# 大模型中的分词方法
NLP任务与一般的机器学习任务有所不同，需要处理的是文本数据，不可以直接输入进深度学习模型，因此在预训练大模型的步骤中，除了
首先收集数据（包括网页数据，代码，论文arxiv，百度百科等）以外，训练一个分词器模型，即如何将我们的文本合理的分词，这样对于输入的每一段文本，我们都可以将其映射到数值空间内（token_id），最后结合ROPE旋转位置编码进行encoding输入给我们的模型，通过transformer架构中的自注意力机制可以有效地处理长文本序列，下面我将参考：[大模型中的分词器tokenizer：BPE、WordPiece、Unigram LM、SentencePiece](https://zhuanlan.zhihu.com/p/620508648)这篇文章进行总结。

根据不同的粒度区分，常见的分词方法有：
- word base:以词为单位，例如：Today is sunday 按照这种分词方法会被分为：[Today ,is,sunday]
- character base:以字符为单位，例如： Today is sunday 按照这种分词方法会被分为：[T,o,d,a,y,i,s,u，n，d，a，y， .]
- subword base:按照词的subword进行分词。如英文Today is sunday. 则会分割成[To， day，is , s，un，day， .]

自GPT2开始，大模型中的常见分词方式为第三种，即以子词的方式进行分词，这里介绍当前预训练大模型中常见的分词方法：Byte Pair Encoding（BPE）


# BPE算法原理
BPE（Byte Pair Encoding）算法是一种数据压缩算法，但在自然语言处理中，它被广泛用于子词单元（subword units）的生成，特别是在处理大型模型的分词训练时。BPE算法通过将常见的字符或字符序列合并成新的单元，从而生成一个词汇表，这个词汇表可以包含从单个字符到完整单词的各种长度的单元。

以下是BPE算法的基本步骤：

1. 初始化：
将词汇表中的每个字符视为一个单独的单元。
对于给定的文本数据，统计每个单元（即字符）的出现频率。
2. 统计频率：
遍历文本数据，统计所有相邻单元对（例如字符对）的出现次数。
3. 合并最频繁的单元对：
选择出现次数最多的单元对进行合并，形成一个新的单元。
更新词汇表，将新单元加入，并删除原来的两个单元。
更新所有包含这两个单元的统计信息，用新单元替换它们。
4. 迭代：
重复步骤2和3，直到达到预设的词汇表大小或迭代次数。
生成词汇表：
在完成所有迭代后，得到的词汇表包含了从单个字符到较长子词单元的各种长度的单元。
5. 编码：
使用生成的词汇表对新的文本数据进行编码。这通常意味着将文本拆分成词汇表中的单元序列。

BPE算法的优点在于：

它能够处理未知单词（OOV，Out-of-Vocabulary words），因为即使一个完整的单词不在词汇表中，它的子词单元也可能在词汇表中。
它生成的词汇表大小可控，可以根据需要进行调整。
相对于基于规则的分词方法，BPE更加灵活，能够适应不同语言的特点。

然而，BPE算法也有一些局限性：

它可能会生成一些在语言学上无意义的子词单元。
对于某些语言，可能需要更多的上下文信息来正确地进行分词。
BPE算法在诸如BERT、GPT等大型语言模型中得到了广泛应用，特别是在处理大规模文本数据和多语言场景时。这些模型通常使用BPE或类似的算法（如SentencePiece、Unigram Language Model等）来生成子词单元，从而有效地处理文本数据。


In [3]:
# 假设有一个汉字字符串
chinese_text = "好"

# 将汉字字符串编码为UTF-8字节串
byte_representation = chinese_text.encode('utf-8')

# 输出字节表示
print(byte_representation)

b'\xe5\xa5\xbd'


# BPE算法实现---python

In [10]:
import re, collections  # 导入需要的库：re 用于正则表达式操作，collections 用于创建字典

def get_vocab(filename):
    vocab = collections.defaultdict(int)  # 创建一个默认值为0的字典 vocab，用于存储词汇表
    with open(filename, 'r', encoding='utf-8') as fhand:  # 打开指定文件以读取内容
        for line in fhand:  # 遍历文件的每一行
            words = line.strip().split()  # 去除行首尾空格并按空格分割行中的单词
            for word in words:  # 遍历每个单词
                vocab[' '.join(list(word)) + ' </w>'] += 1  # 将单词拆分为字符列表，用空格连接字符并添加结束符 '</w>'，更新词频统计
    return vocab  # 返回生成的词汇表字典
def get_stats(vocab):
    pairs = collections.defaultdict(int)  # 创建一个默认值为0的字典 pairs，用于存储字符对的频率统计
    for word, freq in vocab.items():  # 遍历词汇表中的每个单词及其频率
        symbols = word.split()  # 将单词拆分为字符列表
        for i in range(len(symbols)-1):  # 遍历字符列表中除最后一个字符外的每对相邻字符
            pairs[symbols[i], symbols[i+1]] += freq  # 统计相邻字符对的频率
    return pairs  # 返回字符对的频率统计字典

def merge_vocab(pair, v_in):
    v_out = {}  # 创建一个空字典，用于存储合并后的词汇表
    bigram = re.escape(' '.join(pair))  # 将字符对连接成一个大字符，用于正则表达式匹配
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')  # 编译正则表达式，用于在词汇表中查找字符对
    for word in v_in:  # 遍历输入的词汇表
        w_out = p.sub(''.join(pair), word)  # 使用字符对替换词汇表中的字符对
        v_out[w_out] = v_in[word]  # 将替换后的单词及其频率添加到输出词汇表中
    return v_out  # 返回合并后的词汇表

def get_tokens(vocab):
    tokens = collections.defaultdict(int)  # 创建一个默认值为0的字典 tokens，用于存储单词中的各个 token 的频率
    for word, freq in vocab.items():  # 遍历词汇表中的每个单词及其频率
        word_tokens = word.split()  # 将单词拆分为 token
        for token in word_tokens:  # 遍历单词中的每个 token
            tokens[token] += freq  # 统计每个 token 的频率
    return tokens  # 返回包含 token 频率的字典


In [15]:
# 举例-----以便清楚内部实现细节
vocab_dict = collections.defaultdict(int)
with open('vocab.txt', 'r', encoding='utf-8') as fhand:
    for line in fhand:
        words = line.strip().split()
        for word in words:
            vocab_dict[' '.join(list(word)) + ' </w>'] +=1
pairs = collections.defaultdict(int)  # 创建一个默认值为0的字典 pairs，用于存储字符对的频率统计
for word, freq in vocab_dict.items():  # 遍历词汇表中的每个单词及其频率
    symbols = word.split()  # 将单词拆分为字符列表
    for i in range(len(symbols)-1):  # 遍历字符列表中除最后一个字符外的每对相邻字符
        pairs[symbols[i], symbols[i+1]] += freq  # 统计相邻字符对的频率

In [13]:
pairs

defaultdict(int,
            {('\ufeff', 'T'): 1,
             ('T', 'h'): 2,
             ('h', 'e'): 17,
             ('e', '</w>'): 29,
             ('P', 'r'): 6,
             ('r', 'o'): 8,
             ('o', 'j'): 2,
             ('j', 'e'): 2,
             ('e', 'c'): 5,
             ('c', 't'): 3,
             ('t', '</w>'): 16,
             ('G', 'u'): 2,
             ('u', 't'): 6,
             ('t', 'e'): 14,
             ('e', 'n'): 5,
             ('n', 'b'): 3,
             ('b', 'e'): 5,
             ('e', 'r'): 11,
             ('r', 'g'): 4,
             ('g', '</w>'): 5,
             ('e', 'B'): 3,
             ('B', 'o'): 3,
             ('o', 'o'): 9,
             ('o', 'k'): 5,
             ('k', '</w>'): 5,
             ('o', 'f'): 7,
             ('f', '</w>'): 6,
             ('A', 'l'): 2,
             ('l', 'l'): 5,
             ('l', '</w>'): 3,
             ('A', 'r'): 2,
             ('o', 'u'): 7,
             ('u', 'n'): 4,
             ('n', 'd'): 7,
   

In [17]:
# 找到当前相邻字符对出现次数最多的
max_pairs = max(pairs,key=pairs.get)
max_pairs

('e', '</w>')

In [20]:
v_out = {}  # 创建一个空字典，用于存储合并后的词汇表
bigram = re.escape(' '.join(max_pairs))  # 将字符对连接成一个大字符，用于正则表达式匹配
p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')  # 编译正则表达式，用于在词汇表中查找字符对
for word in vocab_dict:  # 遍历输入的词汇表
    w_out = p.sub(''.join(max_pairs), word)  # 使用字符对替换词汇表中的字符对
    print(w_out)
    v_out[w_out] = vocab_dict[word]  # 将替换后的单词及其频率添加到输出词汇表中


 T h e</w>
P r o j e c t </w>
G u t e n b e r g </w>
e B o o k </w>
o f </w>
A l l </w>
A r o u n d </w>
t h e</w>
M o o n </w>
T h i s </w>
e b o o k </w>
i s </w>
f o r </w>
u s e</w>
a n y o n e</w>
a n y w h e r e</w>
i n </w>
U n i t e d </w>
S t a t e s </w>
a n d </w>
m o s t </w>
o t h e r </w>
p a r t s </w>
w o r l d </w>
a t </w>
n o </w>
c o s t </w>
w i t h </w>
a l m o s t </w>
r e s t r i c t i o n s </w>
w h a t s o e v e r . </w>
Y o u </w>
m a y </w>
c o p y </w>
i t , </w>
g i v e</w>
i t </w>
a w a y </w>
o r </w>
r e - u s e</w>
u n d e r </w>
t e r m s </w>
L i c e n s e</w>
i n c l u d e d </w>
t h i s </w>
o n l i n e</w>
w w w . g u t e n b e r g . o r g . </w>
I f </w>
y o u </w>
a r e</w>
n o t </w>
l o c a t e d </w>
S t a t e s , </w>
w i l l </w>
h a v e</w>
t o </w>
c h e c k </w>
l a w s </w>
c o u n t r y </w>
w h e r e</w>
b e f o r e</w>
u s i n g </w>
e B o o k . </w>
T i t l e : </w>
A u t h o r : </w>
J u l e s </w>
V e r n e</w>
T r a n s l a t o 

In [22]:
vocab_dict

defaultdict(int,
            {'\ufeff T h e </w>': 1,
             'P r o j e c t </w>': 2,
             'G u t e n b e r g </w>': 2,
             'e B o o k </w>': 1,
             'o f </w>': 5,
             'A l l </w>': 2,
             'A r o u n d </w>': 2,
             't h e </w>': 12,
             'M o o n </w>': 2,
             'T h i s </w>': 1,
             'e b o o k </w>': 2,
             'i s </w>': 1,
             'f o r </w>': 1,
             'u s e </w>': 1,
             'a n y o n e </w>': 1,
             'a n y w h e r e </w>': 1,
             'i n </w>': 2,
             'U n i t e d </w>': 2,
             'S t a t e s </w>': 1,
             'a n d </w>': 4,
             'm o s t </w>': 1,
             'o t h e r </w>': 1,
             'p a r t s </w>': 1,
             'w o r l d </w>': 1,
             'a t </w>': 4,
             'n o </w>': 2,
             'c o s t </w>': 1,
             'w i t h </w>': 2,
             'a l m o s t </w>': 1,
             'r e s t r i

In [21]:
v_out

{'\ufeff T h e</w>': 1,
 'P r o j e c t </w>': 2,
 'G u t e n b e r g </w>': 2,
 'e B o o k </w>': 1,
 'o f </w>': 5,
 'A l l </w>': 2,
 'A r o u n d </w>': 2,
 't h e</w>': 12,
 'M o o n </w>': 2,
 'T h i s </w>': 1,
 'e b o o k </w>': 2,
 'i s </w>': 1,
 'f o r </w>': 1,
 'u s e</w>': 1,
 'a n y o n e</w>': 1,
 'a n y w h e r e</w>': 1,
 'i n </w>': 2,
 'U n i t e d </w>': 2,
 'S t a t e s </w>': 1,
 'a n d </w>': 4,
 'm o s t </w>': 1,
 'o t h e r </w>': 1,
 'p a r t s </w>': 1,
 'w o r l d </w>': 1,
 'a t </w>': 4,
 'n o </w>': 2,
 'c o s t </w>': 1,
 'w i t h </w>': 2,
 'a l m o s t </w>': 1,
 'r e s t r i c t i o n s </w>': 1,
 'w h a t s o e v e r . </w>': 1,
 'Y o u </w>': 1,
 'm a y </w>': 1,
 'c o p y </w>': 1,
 'i t , </w>': 1,
 'g i v e</w>': 1,
 'i t </w>': 2,
 'a w a y </w>': 1,
 'o r </w>': 2,
 'r e - u s e</w>': 1,
 'u n d e r </w>': 1,
 't e r m s </w>': 1,
 'L i c e n s e</w>': 1,
 'i n c l u d e d </w>': 1,
 't h i s </w>': 2,
 'o n l i n e</w>': 1,
 'w w w . g u t e

In [24]:
pairs1 = get_stats(v_out)
pairs1_ = max(pairs1,key=pairs1.get)

In [25]:
pairs1_

('t', 'h')

In [11]:
# 以pg16457.txt为例
vocab = get_vocab('vocab.txt')

print('=========='*5)
print('Tokens Before BPE')
tokens = get_tokens(vocab)
print('Tokens: {}'.format(tokens))
print('Number of tokens: {}'.format(len(tokens)))
print('=========='*5)

Tokens Before BPE
Tokens: defaultdict(<class 'int'>, {'\ufeff': 1, 'T': 14, 'h': 31, 'e': 85, '</w>': 158, 'P': 7, 'r': 44, 'o': 59, 'j': 4, 'c': 16, 't': 68, 'G': 4, 'u': 27, 'n': 42, 'b': 11, 'g': 15, 'B': 5, 'k': 6, 'f': 12, 'A': 11, 'l': 27, 'd': 30, 'M': 4, 'i': 34, 's': 30, 'a': 46, 'y': 12, 'w': 19, 'U': 6, 'S': 3, 'm': 7, 'p': 9, 'v': 5, '.': 9, 'Y': 1, ',': 6, '-': 1, 'L': 4, 'I': 1, ':': 9, 'J': 2, 'V': 1, 'E': 8, 'R': 6, '6': 2, '2': 4, '0': 4, '5': 2, '[': 1, '#': 1, '1': 2, '4': 1, '7': 1, ']': 1, 'D': 4, 'C': 2, 'K': 3, 'O': 9, '/': 4, '*': 6, 'F': 1, 'H': 2, 'N': 3})
Number of tokens: 63


In [14]:
# 开始合并
num_merges = 10
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens = get_tokens(vocab)
    print('Tokens: {}'.format(tokens))
    print('Number of tokens: {}'.format(len(tokens)))
    print('==========')

Iter: 0
Best pair: ('e', '</w>')
Tokens: defaultdict(<class 'int'>, {'\ufeff': 1, 'T': 14, 'h': 31, 'e</w>': 29, 'P': 7, 'r': 44, 'o': 59, 'j': 4, 'e': 56, 'c': 16, 't': 68, '</w>': 129, 'G': 4, 'u': 27, 'n': 42, 'b': 11, 'g': 15, 'B': 5, 'k': 6, 'f': 12, 'A': 11, 'l': 27, 'd': 30, 'M': 4, 'i': 34, 's': 30, 'a': 46, 'y': 12, 'w': 19, 'U': 6, 'S': 3, 'm': 7, 'p': 9, 'v': 5, '.': 9, 'Y': 1, ',': 6, '-': 1, 'L': 4, 'I': 1, ':': 9, 'J': 2, 'V': 1, 'E': 8, 'R': 6, '6': 2, '2': 4, '0': 4, '5': 2, '[': 1, '#': 1, '1': 2, '4': 1, '7': 1, ']': 1, 'D': 4, 'C': 2, 'K': 3, 'O': 9, '/': 4, '*': 6, 'F': 1, 'H': 2, 'N': 3})
Number of tokens: 64
Iter: 1
Best pair: ('t', 'h')
Tokens: defaultdict(<class 'int'>, {'\ufeff': 1, 'T': 14, 'h': 12, 'e</w>': 29, 'P': 7, 'r': 44, 'o': 59, 'j': 4, 'e': 56, 'c': 16, 't': 49, '</w>': 129, 'G': 4, 'u': 27, 'n': 42, 'b': 11, 'g': 15, 'B': 5, 'k': 6, 'f': 12, 'A': 11, 'l': 27, 'd': 30, 'th': 19, 'M': 4, 'i': 34, 's': 30, 'a': 46, 'y': 12, 'w': 19, 'U': 6, 'S': 3, 'm'